##### Copyright 2018 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=Afd8bu4xJOgh).

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/swift/tutorials/protocol_oriented_generics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/protocol_oriented_generics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/protocol_oriented_generics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Protocol Oriented Programming & Generics

This tutorial will go over protocol oriented programming, and different examples of how they can be used with generics in day to day examples.

## Problem with Object Oriented Programming (OOP)

OOP has been taught as the de-facto design pattern that is taught in school and used throughout the development stack - from web frontend, to mobile, to backend, and machine learning with TensorFlow in Python.

However, OOP does come with some drawbacks:
1. Classes are usually reference types which can cause unexpected behavior when modifying state, especially in a multithreaded environment.
2. Multi-class inheritance can introduce complexity, and some languages out there don't support multi-class inheritance (**Swift is one of these languages!**).

This is where protocol oriented programing comes in handy. But first, lets discuss value types in Swift.


## Swift 💖's Value Types!

Swift was built around the idea of embracing value types. For example, `enums` and `structs` in Swift are filled with features like extensions, methods, and properties which can only typically be found in classes in other languages.

Firstly let's look at how enums are similar to classes

In [2]:
enum Color: String {
  case red = "red"
  case green = "green"
  case blue = "blue"
  
  // A computed property. Note that enums cannot contain stored properties.
  var giveHint: String {
    switch self {
      case .red:
        return "Roses are this color."
      case .green:
        return "Grass is this color."
      case .blue:
        return "The ocean is this color."
    }
  }
  
  // An initializer like for classes
  init?(color: String) {
    switch color {
      case "red":
        self = .red
      case "green":
        self = .green
      case "blue":
        self = .blue
      default:
        return nil
    }
  }
}

// Can extend the enum as well!
extension Color {
  // A function 
  func giveHintFunc() -> String {
    return self.giveHint
  }
}

let c: Color = .red
print("Give me a hint for c: \(c.giveHintFunc())")

let invalidColor = Color(color: "orange")
print("is invalidColor nil: \(invalidColor == nil)")

Give me a hint for c: Roses are this color.
is invalidColor nil: true


Now let's look at structs. Notice that we cannot inherit structs, but we can use protocols instead.

In [3]:
struct Car {
  // Can have variables and constants as stored properties
  var color: Color
  let horsePower: Int
  
  // Can have computed properties
  var watts: Float {
    return Float(horsePower) * 745.7
  }
  
  // Can have lazy variables like in classes!
  lazy var titleCaseColorString: String = {
    let colorString = color.rawValue
    return colorString.prefix(1).uppercased() + colorString.lowercased().dropFirst()
  }()
  
  // A function
  func description() -> String {
    return "This is a \(color) car with \(horsePower) horse power!"
  }
  
  // Can create a variety of initializers
  init(color: Color, horsePower: Int) {
    self.color = color
    self.horsePower = horsePower
  }
  
  // Can define extra initializers other than the default one
  init?(color: String, horsePower: Int) {
    guard let enumColor = Color(color: color) else {
      return nil
    }
    self.color = enumColor
    self.horsePower = horsePower
  }
}

var car = Car(color: .red, horsePower: 250)
print(car.description())
print("Horse power in watts: \(car.watts)")
print(car.titleCaseColorString)

This is a red car with 250 horse power!
Horse power in watts: 186425.0
Red


Finally let's see how they are pass by value types unlike classes:

In [4]:
// Notice we have no problem modifying a constant class with variable properties
class A {
  var a = "a"
}

func foo(_ a: A) {
  a.a = "foo"
}
let a = A()
print(a.a)
foo(a)
print(a.a)

/* 
Uncomment the following code to see how an error is thrown.
Structs are implicitly passed by value, so we cannot modify it.
> "error: cannot assign to property: 'car' is a 'let' constant"
*/

// func modify(car: Car, toColor color: Color) -> Void {
//   car.color = color
// }

// car = Car(color: .red, horsePower: 250)
// print(car.description())
// modify(car: &car, toColor: .blue)
// print(car.description())


a
foo


## So, What are Protocols Exactly?

Simply put, you could consider protocols as a way to introduce multiple inheritence, interfaces, and abstract class concepts to not just classes, but structs and enums as well. 

Let's start by creating protocols for different homes.

In [0]:
protocol House {
  var color: Color { get set }
  var yearBuilt: Int { get }
  var price: Int { get }
  var doIHaveALawn: Bool { get }
}

protocol Apartment {
  var floorLevel: Int { get }
}

protocol Lawn {
  var lawnSize: Int { get }
}

In a object oriented world (with no multiple inheritence), you may have made `House` a base class then used class inheritence to make `Apartment` inherit from `House`. However, here both are completely separate protocols with **zero** coupling! This makes the entire system more flexible in how you design it. 

Let's define an apartment in a major city like San Francisco:

In [0]:
struct SFApartment : House, Apartment {
  var color: Color // Needs to be a var since House has a getter and setter
  let yearBuilt: Int
  let floorLevel: Int
  let doIHaveALawn = false
  
  var price: Int {
    return yearBuilt * -5 + floorLevel * 10000 + 1250000
  }
}

var apartment = SFApartment(color: .red, yearBuilt: 1970, floorLevel: 10)

This specifies a new struct `SFApartment` that conforms to both protocols `House` and `Apartment`.

Now, let's define a house that isn't an apartment, like a bungalow in Los Angeles which have fresh green lawns.

In [0]:
struct LABungalow : House, Lawn { 
  var color: Color
  let yearBuilt: Int
  let squareFootage: Int
  let doIHaveALawn = true
  let lawnSize: Int
  
  var price: Int {
    return yearBuilt * -7 + squareFootage * 5000 + 2330000
  }
}

var bungalow = LABungalow(color: .red, yearBuilt: 1970, squareFootage: 2500, lawnSize: 100)

### Extend Protocols with Default Behaviors

Notice that a bungalow is not an apartment, so we don't conform to the `Apartment`  protocol. Additionally, we are free to add any additional functions and properties like `squareFootage` which isn't a part of the `House` protocol.

What you can notice from the examples is that we have some redundancy. Every time we create a new home, we need to set `doIHaveALawn` to true or false. But we can argue that only homes that conform to the `Lawn` protocol can have a lawn

This is where extensions in Swift can come in handy:

In [0]:
extension House {
  var doIHaveALawn: Bool { return self is Lawn }
}

So now, any class conforming to `House` will have `doIHaveALawn` property defined without any additional work.

![Protocol Comic](https://koenig-media.raywenderlich.com/uploads/2015/06/protocols-extend.png  =350x250)

Thanks to [Ray Wenderlich](https://www.raywenderlich.com/814-introducing-protocol-oriented-programming-in-swift-3) for the comic!

### Overriding Default Behavior

Thus, we have just been able to decorate classes, structs, and enums with unique, and default behavior without introducing any additional state unlike OOP with base classes.

However, if we want to, we can still override the default behavior. Let's say that all expensive apartments with a price greater than  $12,000,000 have lawns.

In [0]:
struct ExpensiveApartment : House, Apartment {
  var color: Color
  let yearBuilt: Int
  let floorLevel: Int
  
  var price: Int {
    return yearBuilt * -1 + floorLevel * 200000 + 9250000
  }
}

extension ExpensiveApartment {
  var doIHaveALawn: Bool {
    return price > 12000000
  }
}

## Standard Library Uses of Protocols

Now that we have an idea how protocols in Swift work, let's go through some typical examples of using the standard library protocols.

### Extending the Standard Library
Lets see how we can add additional functionality to types that exist in Swift already. Since types in Swift aren't built in, but are part of the standard library as structs, this is easy to do.

Let's add the ability to capitalize a sentence which is missing in Swift:

In [10]:
extension String {
  func capitalizeSingleSentence() -> String {
    return prefix(1).uppercased() + self.dropFirst()
  }
}

let sentence = "hello world!"
print("Before: \(sentence)")
print("After: \(sentence.capitalizeSingleSentence())")

Before: hello world!
After: Hello world!


For a more complex example, let's try and do binary search on an array of elements, while also making sure to check that the array is sorted.

In [11]:
extension Collection where Element: Comparable {
  
  // Verify that a Collection is sorted
  func isSorted(_ order: (Element, Element) -> Bool) -> Bool {
    var i = index(startIndex, offsetBy: 1)
    
    while i < endIndex {
      // The longer way of calling a binary function like <, <=, ==, etc.
      if !order(self[index(i, offsetBy: -1)], self[i]) {
        return false
      }
      i = index(i, offsetBy: 1)
    }
    return true
  }
  
  // Perform binary search on a Collection, verifying it is sorted
  func binarySearch(_ num: Element) -> Index? {
    guard self.isSorted(<=) else {
      return nil
    }
    var low = startIndex
    var high = endIndex
    
    while low <= high {
      let mid = index(low, offsetBy: distance(from: low, to: high)/2)

      if self[mid] == num {
        return mid
      } else if self[mid] < num {
        low = index(mid, offsetBy: 1)
      } else {
        high = index(mid, offsetBy: -1)
      }
    }
    
    return nil
  }
}

print([2, 2, 5, 7, 11, 13, 17].binarySearch(5)!)
print(["a", "b", "c", "d"].binarySearch("b")!)
print([1.1, 2.2, 3.3, 4.4, 5.5].binarySearch(3.3)!)

2
1
2


We do this by extending the [`Collection`](https://developer.apple.com/documentation/swift/collection) protocol which defines *"a sequence whose elements can be traversed multiple times, nondestructively, and accessed by an indexed subscript."* Since arrays can be indexed using the square bracket notation, this is the protocol we want to extend.

Similarly, we only want to add this utility function to arrays whose elements can be compared. This is the reason why we have `where Element: Comparable`. 

The `where` clause is a part of Swift's generic type system, which we will cover soon, but in short lets us add additional requirements to the extension we are writing, such as to require the type to implement a protocol, to require two types to be the same, or to require a class to have a particular superclass.

[`Element`](https://developer.apple.com/documentation/swift/sequence/2908099-element) is the associated type (type alias) of the element in the `Collection` conforming value type. `Element` is defined within the [`Sequence`](https://developer.apple.com/documentation/swift/sequence) protocol, but since `Collection` conforms to `Sequence`, it inherits the `Element` associated type.

[`Comparable`](https://developer.apple.com/documentation/swift/comparable) is a protocol that defines *"a type that can be compared using the relational operators <, <=, >=, and >."*. Since we are performing binary search on a sorted list, this of course has to be true or else we don't know whether to recurse/iterate left or right in the binary search.

As a side note about the implementation,  for more info on the `index(_:offsetBy:)` function that was used, refer to the following [documentation](https://developer.apple.com/documentation/swift/string/1786175-index).

## Generics + Protocols = 💥
Generics and protocols can be a powerful tool if used correctly to avoid duplicate code, but still keeping it very expressive.

Firstly, look over our other tutorial, [A Swift Tour](https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/a_swift_tour.ipynb), which briefly covers generics at the end of the Colab book.

Assuming you have a general idea about generics, let's quickly take a look at some advanced uses before moving on to how protocol oriented programming is used in Swift for TensorFlow which really exploits the combined power of protocol oriented programming and generics!

When a single type has multiple requirements such as a type adhering to several protocols, you have several options at your disposal:

In [12]:
typealias numComp = Comparable & FloatingPoint

func foo1<T: numComp>(a: T, b: T) -> Bool {
  return a > b
}

func foo2<T: Comparable & FloatingPoint>(a: T, b: T) -> Bool {
  return a > b
}

func foo3<T>(a: T, b: T) -> Bool 
  where T: numComp
{
  return a > b
}

func foo4<T>(a: T, b: T) -> Bool 
  where T: Comparable & FloatingPoint
{
  return a > b
}

func foo5<T: FloatingPoint>(a: T, b: T) -> Bool 
  where T: Comparable
{
  return a > b
}

print(foo1(a: 1, b: 2))
print(foo2(a: 1, b: 2))
print(foo3(a: 1, b: 2))
print(foo4(a: 1, b: 2))
print(foo5(a: 1, b: 2))

false
false
false
false
false


Notice the use of `typealias` at the top. This adds a named alias of an existing type into your program. After a type alias is declared, the aliased name can be used instead of the existing type everywhere in your program. The existing type can be a named type or a compound type. Type aliases do not create new types; they simply allow a name to refer to an existing type.

Now let's see how we can use protocols and generics together.

Let's imagine we are a computer store with the following requirements on any laptop we sell for determining how we organize them in the back of the store:

In [0]:
enum Box {
  case small
  case medium
  case large
}

enum Mass {
  case light
  case medium
  case heavy
}

// NOTE: CustomStringConvertible protocol lets us pretty print the object
// conforming to Laptop
struct Laptop : CustomStringConvertible {
  var name: String
  var box: Box
  var mass: Mass
  
  var description: String {
    return "(\(self.name) \(self.box) \(self.mass))"
  }
}

However, we got a new requirement of grouping our Laptops by mass since the shelves have weight restrictions.

In [14]:
func filterLaptops(_ array: [Laptop], by mass: Mass) -> [Laptop] {
  return array.filter{ $0.mass == mass }
}

let listOfLaptops: [Laptop] = [
  Laptop(name: "a", box: .small, mass: .light),
  Laptop(name: "b", box: .large, mass: .medium),
  Laptop(name: "c", box: .medium, mass: .heavy),
  Laptop(name: "d", box: .large, mass: .light)
]

let filteredList = filterLaptops(listOfLaptops, by: .light)
print(filteredList)

[(a small light), (d large light)]


However, what if we wanted to filter by something other than `Mass`? We can do the following naive approach.

In [15]:
// Define a protocol which will act as our comparator
protocol FilterPredicate {
  associatedtype Device
  func shouldKeep(item: Device) -> Bool
}

// =============================================================================
// Define the structs we will use for passing into our filtering function
struct BoxFilter : FilterPredicate {
  typealias Device = Laptop
  var box: Box 
  
  func shouldKeep(item: Laptop) -> Bool {
    return item.box == box
  }
}

struct MassFilter : FilterPredicate {
  typealias Device = Laptop  
  var mass: Mass
  
  func shouldKeep(item: Laptop) -> Bool {
    return item.mass == mass
  }
}

// =============================================================================
// Make sure our filter conforms to FilterPredicate and that we are filtering
// Laptops
func filterLaptops2<F: FilterPredicate>
(
  _ array: [Laptop], 
  by filter: F
) -> [Laptop] where Laptop == F.Device {
  return array.filter{ filter.shouldKeep(item: $0) }
}

// =============================================================================

print(filterLaptops2(listOfLaptops, by: BoxFilter(box: .large)))
print(filterLaptops2(listOfLaptops, by: MassFilter(mass: .heavy)))

[(b large medium), (d large light)]
[(c medium heavy)]


Awesome! Now we are able to filter based on anything. However, we are only able to filter `Laptop`s. What about being able to filter anything that is in a box and has mass? Maybe this warehouse of laptops will also be used for servers which have a different customer base.

In [16]:
// Define 2 new protocols so we can filter anything in a box and which has mass
protocol HasMass {
  var mass: Mass { get }
}

protocol HasBox {
  var box: Box { get }
}

// =============================================================================
// Define the new Laptop and Server struct which have mass and a box
struct Laptop2 : CustomStringConvertible, HasBox, HasMass {
  var name: String
  var box: Box
  var mass: Mass
  
  var description: String {
    return "(\(self.name) \(self.box) \(self.mass))"
  }
}

struct Server : CustomStringConvertible, HasBox, HasMass {
  var isWorking: Bool
  var name: String
  let box: Box
  let mass: Mass

  var description: String {
    if isWorking {
      return "(working \(self.name) \(self.box) \(self.mass))"
    } else {
      return "(notWorking \(self.name) \(self.box) \(self.mass))"
    }
  }
}

// =============================================================================
// Define the structs we will use for passing into our filtering function
struct BoxFilter2<T: HasBox> : FilterPredicate {
  var box: Box 
  
  func shouldKeep(item: T) -> Bool {
    return item.box == box
  }
}

struct MassFilter2<T: HasMass> : FilterPredicate {
  var mass: Mass
  
  func shouldKeep(item: T) -> Bool {
    return item.mass == mass
  }
}

// =============================================================================
// Define the new filter function
func filterAnything<F: FilterPredicate, T>
(
  _ array: [T], 
  by filter: F
) -> [T] where T == F.Device {
  return array.filter{ filter.shouldKeep(item: $0) }
}

// =============================================================================
// Let's test the function out!
let servers: [Server] = [
  Server(isWorking: true, name: "serverA", box: .small, mass: .heavy),
  Server(isWorking: false, name: "serverB", box: .medium, mass: .medium),
  Server(isWorking: true, name: "serverC", box: .large, mass: .light),
  Server(isWorking: false, name: "serverD", box: .medium, mass: .light),
  Server(isWorking: true, name: "serverE", box: .small, mass: .heavy)
]

let products: [Laptop2] = [
  Laptop2(name: "a", box: .small, mass: .light),
  Laptop2(name: "b", box: .large, mass: .medium),
  Laptop2(name: "c", box: .medium, mass: .heavy),
  Laptop2(name: "d", box: .large, mass: .light)
]

print(filterAnything(servers, by: BoxFilter2(box: .small)))
print(filterAnything(servers, by: MassFilter2(mass: .medium)))

print(filterAnything(products, by: BoxFilter2(box: .small)))
print(filterAnything(products, by: MassFilter2(mass: .medium)))

[(working serverA small heavy), (working serverE small heavy)]
[(notWorking serverB medium medium)]
[(a small light)]
[(b large medium)]


We have now been able to filter an array by not only any property of a specific `struct`, but also be able to filter any struct which has that property!

We have been able to use generics in a protocol oriented way to achieve a high level of abstraction.

# Protocols in Swift for TensorFlow

Every time you will be creating something with Swift for TensorFlow, you will likely be using a protocol. We will go over a typical protocol you would use when creating a model - the `Layer` protocol. But for the full list of all protocols used in Swift for TensorFlow, refer to the documentation [here](https://www.tensorflow.org/swift/api_docs/Protocols).

## Creating a Simple Model Using the `Layer` Protocol
This model was taken from our [Model Training Walkthrough](https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/model_training_walkthrough.ipynb) Colab.

As can be seen [here](https://www.tensorflow.org/swift/api_docs/Protocols/Layer), to conform to the `Layer` protocol, we need to define the function `call(_:)` which is the output of our model given some input.

In [0]:
import TensorFlow

let hiddenSize: Int = 10
struct IrisModel: Layer {
  var layer1 = Dense<Float>(inputSize: 4, outputSize: hiddenSize, activation: relu)
  var layer2 = Dense<Float>(inputSize: hiddenSize, outputSize: hiddenSize, activation: relu)
  var layer3 = Dense<Float>(inputSize: hiddenSize, outputSize: 3)

  @differentiable
  func call(_ input: Tensor<Float>) -> Tensor<Float> {
    return input.sequenced(through: layer1, layer2, layer3)
  }
}

And just like that we have  model by using the `Layer` protocol!

# More Protocol Oriented Programming Resources
Here's additional resources on the topics discussed:
- [WWDC 2015: Protocol-Oriented Programming in Swift](https://developer.apple.com/videos/play/wwdc2015/408/): this was presented using Swift 2, so a lot has changed since then (e.g. name of the protocols they used in the presentation) but is still a good resource for the theory and use behind it.
- [Introducing Protocol-Oriented Programming in Swift 3](https://www.raywenderlich.com/814-introducing-protocol-oriented-programming-in-swift-3): again written in Swift 3, so some of the code may need to be modified in order to have it compile successfully, but another great resource.
- [Generics](https://docs.swift.org/swift-book/LanguageGuide/Generics.html): Swifts own documentation for Swift 5 all about generics.